<a href="https://colab.research.google.com/github/Santosh-Gupta/NaturalLanguageRecommendations/blob/srihari-dev/notebooks/model_debug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers tqdm --quiet
%tensorflow_version 2.x

You should consider upgrading via the 'pip install --upgrade pip' command.


UsageError: Line magic function `%tensorflow_version` not found.


In [11]:
import os
from tqdm.notebook import tqdm
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Lambda, Dense, Activation, Concatenate
from transformers import TFBertModel
print('TensorFlow:', tf.__version__)

TensorFlow: 2.0.0


In [5]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.MirroredStrategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

W1224 01:37:50.389210 4672177600 cross_device_ops.py:1209] There is non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.


REPLICAS:  1


In [6]:
batch_size = 12 * strategy.num_replicas_in_sync
embedding_dim = 512
autotune = tf.data.experimental.AUTOTUNE

In [7]:
def get_random_title():
    return tf.random.uniform(shape=[512], maxval=200, dtype=tf.int32)

def get_random_citation():
    vector = tf.random.uniform(shape=[embedding_dim], minval=-1, maxval=1, dtype=tf.float32)
    normed_vector = tf.math.l2_normalize(vector)
    return normed_vector

def generate_sample():
    title = get_random_title()
    posCitations = get_random_citation()
    return title, posCitations

In [8]:
class TFrecordWriter:
    def __init__(self,
                 n_samples,
                 n_shards,
                 output_dir='',
                 prefix=''):
        self.n_samples = n_samples
        self.n_shards = n_shards
        self.step_size = self.n_samples//self.n_shards + 1
        self.prefix = prefix
        self.output_dir = output_dir
        self.buffer = []
        self.file_count = 1
        
    def make_example(self, title, vector):
        feature = {
            'title': tf.train.Feature(int64_list=tf.train.Int64List(value=title)),
            'citation': tf.train.Feature(float_list=tf.train.FloatList(value=vector))
        }
        return tf.train.Example(features=tf.train.Features(feature=feature))
        
    def write_tfrecord(self, tfrecord_path):
        print('writing {} samples in {}'.format(len(self.buffer), tfrecord_path))
        with tf.io.TFRecordWriter(tfrecord_path) as writer:
            for (title, vector) in tqdm(self.buffer):
                example = self.make_example(title, vector)
                writer.write(example.SerializeToString())
    
    def push(self, title, vector):
        self.buffer.append([title, vector])
        if len(self.buffer) == self.step_size:
            fname = self.prefix + '_000' + str(self.file_count) + '.tfrecord'
            tfrecord_path = os.path.join(self.output_dir, fname)
            self.write_tfrecord(tfrecord_path)
            self.clear_buffer()
            self.file_count += 1
            
    def flush_last(self):
        if len(self.buffer):
            fname = self.prefix + '_000' + str(self.file_count) + '.tfrecord'
            tfrecord_path = os.path.join(self.output_dir, fname)
            self.write_tfrecord(tfrecord_path)
            
    def clear_buffer(self):
        self.buffer = []

In [12]:
!mkdir 'tfrecords'
tfrecord_writer = TFrecordWriter(1000, 6, 'tfrecords', 'train')

for i in range(1000):
    title, vector = generate_sample()
    tfrecord_writer.push(title, vector)
tfrecord_writer.flush_last()

mkdir: tfrecords: File exists
writing 167 samples in tfrecords/train_0001.tfrecord



writing 167 samples in tfrecords/train_0002.tfrecord



writing 167 samples in tfrecords/train_0003.tfrecord



writing 167 samples in tfrecords/train_0004.tfrecord



writing 167 samples in tfrecords/train_0005.tfrecord



writing 165 samples in tfrecords/train_0006.tfrecord


In [13]:
base_dir = 'tfrecords'
tfrecords_pattern = os.path.join(base_dir, 'train*')

In [14]:
features = {
    'title':tf.io.FixedLenFeature([512], dtype=tf.int64),
    'citation':tf.io.FixedLenFeature([512], dtype=tf.float32),
    }

def parse_example(example_proto):
    parsed_example = tf.io.parse_single_example(example_proto, features)
    title = parsed_example['title']
    citation = parsed_example['citation']
    return (title, citation), 1.0

In [15]:
with strategy.scope():
    files = tf.data.Dataset.list_files(tfrecords_pattern)
    dataset = files.interleave(tf.data.TFRecordDataset,
                               cycle_length=16,
                               block_length=4,
                               num_parallel_calls=autotune)
    dataset = dataset.map(parse_example, num_parallel_calls=autotune)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    dataset = dataset.prefetch(autotune)
tf.data.experimental.get_structure(dataset)

((TensorSpec(shape=(12, 512), dtype=tf.int64, name=None),
  TensorSpec(shape=(12, 512), dtype=tf.float32, name=None)),
 TensorSpec(shape=(12,), dtype=tf.float32, name=None))

In [16]:
@tf.function
def loss_fn(_, probs):
    '''
        1. Every sample is its own positive, and  the rest of the
            elements in the batch are its negative.
        2. Each TPU core gets 1/8 * global_batch_size elements, hence
            compute shape dynamically.
        3. Dataset produces dummy labels to make sure the loss_fn matches
            the loss signature of keras, actual labels are computed inside this
            function.
    '''
    bs = tf.shape(probs)[0] 
    labels = tf.eye(bs, bs)
    return tf.losses.categorical_crossentropy(labels, probs)
    
def create_model():
    title = tf.keras.Input(shape=(512,), dtype=tf.int32)    # from bert tokenizer
    citation = tf.keras.Input(shape=(512,))                 # normalized word2vec outputs
    
    bert_model = TFBertModel.from_pretrained('bert-base-uncased')
    
    titleOut = bert_model(title)
    titleOutMean = tf.reduce_mean(titleOut[0], axis=1)
    titleOutSim = Dense(units=embedding_dim, activation='tanh', name='DenseTitle')(titleOutMean)

    citationSim = Dense(units=embedding_dim, activation='tanh', name='DenseCitation')(citation)

    # Get dot product of each of title x citation combinations
    dotProduct = tf.reduce_sum(tf.multiply(titleOutSim[:, None, :], citationSim), axis=-1)
    
    # Softmax to make sure each row has sum == 1.0
    probs = tf.nn.softmax(dotProduct, axis=-1)

    model = tf.keras.Model(inputs=[title, citation], outputs=[probs])
    return model

In [17]:
with strategy.scope():
    model = create_model()
    model.compile(loss=loss_fn, optimizer=tf.optimizers.Adam())

I1224 01:53:04.251464 4672177600 modeling_tf_utils.py:339] Layers from pretrained model not used in TFBertModel: ['mlm___cls', 'nsp___cls']


In [18]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512)]        0                                            
__________________________________________________________________________________________________
tf_bert_model (TFBertModel)     ((None, 512, 768), ( 109482240   input_1[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mean (TensorFlowOpL [(None, 768)]        0           tf_bert_model[0][0]              
__________________________________________________________________________________________________
DenseTitle (Dense)              (None, 512)          393728      tf_op_layer_Mean[0][0]           
______________________________________________________________________________________________

In [ ]:
model.fit(dataset, epochs=5, steps_per_epoch=20)